In [1]:
import cv2
import numpy as np
from tqdm import tqdm
from time import time
import matplotlib.pyplot as plt
from scipy.signal import medfilt

In [2]:
PIXELS = 16
RADIUS = 300
HORIZONTAL_BORDER = 30
file_name = '../data/small-shaky-5.avi'
cap = cv2.VideoCapture(file_name)
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [3]:
def point_transform(H, pt):
    """
    @param: H is homography matrix of dimension (3x3) 
    @param: pt is the (x, y) point to be transformed
    
    Return:
            returns a transformed point ptrans = H*pt.
    """
    a = H[0,0]*pt[0] + H[0,1]*pt[1] + H[0,2]
    b = H[1,0]*pt[0] + H[1,1]*pt[1] + H[1,2]
    c = H[2,0]*pt[0] + H[2,1]*pt[1] + H[2,2]
    return [a/c, b/c]

In [5]:
def motion_propagate(old_points, new_points, old_frame):
    """
    @param: old_points are points in old_frame that are 
            matched feature points with new_frame
    @param: new_points are points in new_frame that are 
            matched feature points with old_frame
    @param: old_frame is the frame to which 
            motion mesh needs to be obtained
    @param: H is the homography between old and new points
    
    Return:
            returns a motion mesh in x-direction 
            and y-direction for old_frame
    """
    # spreads motion over the mesh for the old_frame
    x_motion = {}; y_motion = {};
    cols, rows = old_frame.shape[1]/PIXELS, old_frame.shape[0]/PIXELS
    
    # pre-warping with global homography
    H, _ = cv2.findHomography(old_points, new_points, cv2.RANSAC)
    for i in range(rows):
        for j in range(cols):
            pt = [PIXELS*j, PIXELS*i]
            ptrans = point_transform(H, pt)
            x_motion[i, j] = pt[0]-ptrans[0]
            y_motion[i, j] = pt[1]-ptrans[1]
            
    # disturbute feature motion vectors
    temp_x_motion = {}; temp_y_motion = {}
    for i in range(rows):
        for j in range(cols):
            vertex = [PIXELS*j, PIXELS*i]
            for pt, st in zip(old_points, new_points):
                
                # velocity = point - feature point match in next frame
                # dst = sqrt((vertex[0]-st[0])**2+(vertex[1]-st[1])**2)
                
                # velocity = point - feature point in current frame
                dst = np.sqrt((vertex[0]-pt[0])**2+(vertex[1]-pt[1])**2)
                if dst < RADIUS:
                    ptrans = point_transform(H, pt)
                    try:
                        temp_x_motion[i, j].append(st[0]-ptrans[0])
                    except:
                        temp_x_motion[i, j] = [st[0]-ptrans[0]]
                    try:
                        temp_y_motion[i, j].append(st[1]-ptrans[1])
                    except:
                        temp_y_motion[i, j] = [st[1]-ptrans[1]]
    
    # apply median filter (f-1) on obtained motion for each vertex
    x_motion_mesh = np.zeros((rows, cols), dtype=float)
    y_motion_mesh = np.zeros((rows, cols), dtype=float)
    for key in x_motion.keys():
        try:
            temp_x_motion[key].sort()
            x_motion_mesh[key] = x_motion[key]+temp_x_motion[key][len(temp_x_motion[key])/2]
        except KeyError:
            x_motion_mesh[key] = x_motion[key]
        try:
            temp_y_motion[key].sort()
            y_motion_mesh[key] = y_motion[key]+temp_y_motion[key][len(temp_y_motion[key])/2]
        except KeyError:
            y_motion_mesh[key] = y_motion[key]
    
    # apply second median filter (f-2) over the motion mesh for outliers
    x_motion_mesh = medfilt(x_motion_mesh, kernel_size=[3, 3])
    y_motion_mesh = medfilt(y_motion_mesh, kernel_size=[3, 3])
    
    return x_motion_mesh, y_motion_mesh

In [6]:
def generate_vertex_profiles(x_paths, y_paths, x_motion_mesh, y_motion_mesh):
    """
    @param: x_paths is vertex profiles along x-direction
    @param: y_paths is vertex profiles along y_direction
    @param: x_motion_mesh is obtained motion mesh along 
            x-direction from motion_propogate()
    @param: y_motion_mesh is obtained motion mesh along 
            y-direction from motion_propogate()

    Returns:
            returns updated x_paths, y_paths with new 
            x_motion_mesh, y_motion_mesh added to the 
            last x_paths, y_paths
    """
    new_x_path = x_paths[:, :, -1] + x_motion_mesh
    new_y_path = y_paths[:, :, -1] + y_motion_mesh
    x_paths = np.concatenate((x_paths, np.expand_dims(new_x_path, axis=2)), axis=2)
    y_paths = np.concatenate((y_paths, np.expand_dims(new_y_path, axis=2)), axis=2)
    return x_paths, y_paths

In [7]:
def gauss(t, r, window_size):
    """
    @param: window_size is the size of window over which gaussian to be applied
    @param: t is the index of current point 
    @param: r is the index of point in window 
    
    Return:
            returns spacial guassian weights over a window size
    """
    return np.exp((-9*(r-t)**2)/window_size**2)

In [8]:
def optimize_path(c, iterations=100, window_size=6):
    """
    @param: c is original camera trajectory
    @param: window_size is the hyper-parameter for the smoothness term
    
    
    Returns:
            returns an optimized gaussian smooth camera trajectory 
    """
    lambda_t = 100
    p = np.empty_like(c)
    
    W = np.zeros((c.shape[2], c.shape[2]))
    for t in range(W.shape[0]):
        for r in range(-window_size/2, window_size/2+1):
            if t+r < 0 or t+r >= W.shape[1] or r == 0:
                continue
            W[t, t+r] = gauss(t, t+r, window_size)

    gamma = 1+lambda_t*np.dot(W, np.ones((c.shape[2],)))
    
    bar = tqdm(total=c.shape[0]*c.shape[1])
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):
            P = np.asarray(c[i, j, :])
            for iteration in range(iterations):
                P = np.divide(c[i, j, :]+lambda_t*np.dot(W, P), gamma)
            p[i, j, :] = np.asarray(P)
            bar.update(1)
    
    bar.close()
    return p

In [9]:
def mesh_warp_frame(frame, x_motion_mesh, y_motion_mesh):
    """
    @param: frame is the current frame
    @param: x_motion_mesh is the motion_mesh to 
            be warped on frame along x-direction
    @param: y_motion_mesh is the motion mesh to 
            be warped on frame along y-direction

    Returns:
            returns a mesh warped frame according 
            to given motion meshes x_motion_mesh, 
            y_motion_mesh
    """
    
    # define handles on mesh in x-direction
    map_x = np.zeros((frame.shape[0], frame.shape[1]), np.float32)
    
    # define handles on mesh in y-direction
    map_y = np.zeros((frame.shape[0], frame.shape[1]), np.float32)
    
    for i in range(x_motion_mesh.shape[0]-1):
        for j in range(x_motion_mesh.shape[1]-1):

            src = [[j*PIXELS, i*PIXELS],
                   [j*PIXELS, (i+1)*PIXELS],
                   [(j+1)*PIXELS, i*PIXELS],
                   [(j+1)*PIXELS, (i+1)*PIXELS]]
            src = np.asarray(src)
            
            dst = [[j*PIXELS+x_motion_mesh[i, j], i*PIXELS+y_motion_mesh[i, j]],
                   [j*PIXELS+x_motion_mesh[i+1, j], (i+1)*PIXELS+y_motion_mesh[i+1, j]],
                   [(j+1)*PIXELS+x_motion_mesh[i, j+1], i*PIXELS+y_motion_mesh[i, j+1]],
                   [(j+1)*PIXELS+x_motion_mesh[i+1, j+1], (i+1)*PIXELS+y_motion_mesh[i+1, j+1]]]
            dst = np.asarray(dst)
            H, _ = cv2.findHomography(src, dst, cv2.RANSAC)
            
            for k in range(PIXELS*i, PIXELS*(i+1)):
                for l in range(PIXELS*j, PIXELS*(j+1)):
                    x = H[0, 0]*l+H[0, 1]*k+H[0, 2]
                    y = H[1, 0]*l+H[1, 1]*k+H[1, 2]
                    w = H[2, 0]*l+H[2, 1]*k+H[2, 2]
                    if not w == 0:
                        x = x/(w*1.0); y = y/(w*1.0)
                    else:
                        x = l; y = k
                    map_x[k, l] = x
                    map_y[k, l] = y
    
    # repeat motion vectors for remaining frame in y-direction
    for i in range(PIXELS*x_motion_mesh.shape[0], map_x.shape[0]):
            map_x[i, :] = map_x[PIXELS*x_motion_mesh.shape[0]-1, :]
            map_y[i, :] = map_y[PIXELS*x_motion_mesh.shape[0]-1, :]
    
    # repeat motion vectors for remaining frame in x-direction
    for j in range(PIXELS*x_motion_mesh.shape[1], map_x.shape[1]):
            map_x[:, j] = map_x[:, PIXELS*x_motion_mesh.shape[0]-1]
            map_y[:, j] = map_y[:, PIXELS*x_motion_mesh.shape[0]-1]
            
    # deforms mesh
    new_frame = cv2.remap(frame, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    return new_frame

In [10]:
start_time = time()    

# generate stabilized video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('../stable.avi', fourcc, frame_rate, (2*frame_width, frame_height))

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 1000,
                    qualityLevel = 0.3,
                    minDistance = 7,
                    blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.03))

# Take first frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# preserve aspect ratio
VERTICAL_BORDER = (HORIZONTAL_BORDER*old_gray.shape[1])/old_gray.shape[0]

In [11]:
# motion meshes in x-direction and y-direction
x_motion_meshes = []; y_motion_meshes = []

# path parameters
x_paths = np.zeros((old_frame.shape[0]/PIXELS, old_frame.shape[1]/PIXELS, 1))
y_paths = np.zeros((old_frame.shape[0]/PIXELS, old_frame.shape[1]/PIXELS, 1))

frame_num = 1
bar = tqdm(total=frame_count)
while frame_num < frame_count:

    # processing frames
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # find corners in it
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # estimate motion mesh for old_frame
    x_motion_mesh, y_motion_mesh = motion_propagate(good_old, good_new, frame)
    try:
        x_motion_meshes = np.concatenate((x_motion_meshes, np.expand_dims(x_motion_mesh, axis=2)), axis=2)
        y_motion_meshes = np.concatenate((y_motion_meshes, np.expand_dims(y_motion_mesh, axis=2)), axis=2)
    except:
        x_motion_meshes = np.expand_dims(x_motion_mesh, axis=2)
        y_motion_meshes = np.expand_dims(y_motion_mesh, axis=2)

    # generate vertex profiles
    x_paths, y_paths = generate_vertex_profiles(x_paths, y_paths, x_motion_mesh, y_motion_mesh)

    # updates frames
    bar.update(1)
    frame_num += 1
    old_frame = frame.copy()
    old_gray = frame_gray.copy()

bar.close()

100%|█████████▉| 209/210 [00:24<00:00,  8.28it/s]


In [12]:
# optimize for smooth vertex profiles
optimization = time()
sx_paths = optimize_path(x_paths)
sy_paths = optimize_path(y_paths)
print 'Time Taken: ', time()-optimization

100%|██████████| 220/220 [00:01<00:00, 127.72it/s]

Time Taken:  2.94431400299


In [13]:
# plot some vertex profiles
for i in range(0, x_paths.shape[0]):
    for j in range(0, x_paths.shape[1], 10):
        plt.plot(x_paths[i, j, :])
        plt.plot(sx_paths[i, j, :])
        plt.savefig('../results/paths/'+str(i)+'_'+str(j)+'.png')
        plt.clf()

In [14]:
# U = P-C
x_motion_meshes = np.concatenate((x_motion_meshes, np.expand_dims(x_motion_meshes[:, :, -1], axis=2)), axis=2)
y_motion_meshes = np.concatenate((y_motion_meshes, np.expand_dims(y_motion_meshes[:, :, -1], axis=2)), axis=2)
new_x_motion_meshes = sx_paths-x_paths
new_y_motion_meshes = sy_paths-y_paths

In [15]:
r = 3
frame_num = 0
bar = tqdm(total=frame_count)
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
while frame_num < frame_count:
    try:
        # reconstruct from frames
        ret, frame = cap.read()
        x_motion_mesh = x_motion_meshes[:, :, frame_num]
        y_motion_mesh = y_motion_meshes[:, :, frame_num]
        new_x_motion_mesh = new_x_motion_meshes[:, :, frame_num]
        new_y_motion_mesh = new_y_motion_meshes[:, :, frame_num]
        
        # mesh warping
        new_frame = mesh_warp_frame(frame, new_x_motion_mesh, new_y_motion_mesh)
        new_frame = new_frame[HORIZONTAL_BORDER:-HORIZONTAL_BORDER, VERTICAL_BORDER:-VERTICAL_BORDER, :]
        new_frame = cv2.resize(new_frame, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_CUBIC)
        output = np.concatenate((frame, new_frame), axis=1)
        out.write(output)

        # draw old motion vectors
        for i in range(x_motion_mesh.shape[0]):
            for j in range(x_motion_mesh.shape[1]):
                theta = np.arctan2(y_motion_mesh[i, j], x_motion_mesh[i, j])
                cv2.line(frame, (j*PIXELS, i*PIXELS), (int(j*PIXELS+r*np.cos(theta)), int(i*PIXELS+r*np.sin(theta))), 1)
        cv2.imwrite('../results/old_motion_vectors/'+str(frame_num)+'.jpg', frame)

        # draw new motion vectors
        for i in range(new_x_motion_mesh.shape[0]):
            for j in range(new_x_motion_mesh.shape[1]):
                theta = np.arctan2(new_y_motion_mesh[i, j], new_x_motion_mesh[i, j])
                cv2.line(new_frame, (j*PIXELS, i*PIXELS), (int(j*PIXELS+r*np.cos(theta)), int(i*PIXELS+r*np.sin(theta))), 1)
        cv2.imwrite('../results/new_motion_vectors/'+str(frame_num)+'.jpg', new_frame)

        frame_num += 1
        bar.update(1)
    except:
        break
bar.close()

100%|██████████| 210/210 [00:35<00:00,  6.14it/s]


In [16]:
cap.release()
out.release()
print 'Time elapsed: ', str(time()-start_time)

Time elapsed:  118.327890873
